## Application of Dictionary Attack Technique

In [1]:
# read names and surnames in 'filename1' and 'filename2'
def read_names(filename1, filename2):
    file1 = codecs.open(filename1,'r','utf-8')
    file2 = codecs.open(filename2,'r','utf-8')
    names = file1.readlines()
    surnames = file2.readlines()
    for i in range (len(names)):
        names[i] = names[i].replace("\r","").replace("\n","")
    for i in range (len(surnames)):
        surnames[i] = surnames[i].replace("\r","").replace("\n","")
    return names, surnames

In [2]:
# for all possible name-surname combinations
# generate a dictionary of pairs (pseudonym,email)
def generate_hashes(filename1, filename2):
    mapping_dir = "data/mapping"
    if len(listdir(mapping_dir))>0:
        return
    names, surnames = read_names(filename1, filename2)
    mapping_dict = {}
    i=0
    j=0
    for name in names:
        for surname in surnames:
            # split the dictionary into smaller pieces
            if i == 500000:
                i=0
                j = j + 1
                with open(mapping_dir + '/mapping'+str(j)+'.pickle', 'wb') as handle:
                    pk.dump(mapping_dict, handle)
                print("File named mapping" + str(j) + " successfully saved")
                mapping_dict.clear()
            else:
                curr_email = (name + '.' + surname).lower().replace("'", "").replace(" ","") + '@studium.unict.it'
                curr_hash = hashlib.sha256(curr_email.encode()).hexdigest()
                mapping_dict.update({curr_hash:curr_email})
                i= i + 1

In [3]:
# find the indentifiers associated to hashes in 'df'
# through a dictionary look-up
def crack_hashes(df):
    mapping_dir = "data/mapping"
    partial_mapping = {}
    for i in range(len(df)):
        curr_hash = df.iloc[i,0]
        for mapping in listdir(mapping_dir):
            mapping_name = mapping_dir+"/"+mapping
            with open(mapping_name, 'rb') as handle:
                curr_mapping = pk.load(handle)
            plain_email = curr_mapping.get(curr_hash)
            if(plain_email is not None):
                partial_mapping.update({curr_hash:plain_email})
                break
    with open('data/partial_mapping.pickle', 'wb') as handle:
        pk.dump(partial_mapping, handle)

In [4]:
import pandas as pd
import hashlib
from os import listdir
import codecs
import pickle as pk

# create the dictionary
generate_hashes('data/nomi_italiani.txt', 'data/cognomi_italiani.txt')

In [5]:
# read emails hash to revert back
df = pd.read_csv("data/filtered_students.csv")
df

,Pseudonym,Passed,Sex,Off-site,Holder
0,6098239d3dff4173859757b08efbcd394444684e87800b...,1,M,1,1
1,f87649a54babb5ece11636e038b86f91d978bfc13042d5...,1,M,1,1
2,ecd1c553df13a400324db32ef72dd652784ee03bbc6cd3...,1,M,1,1


In [6]:
# apply the dictionary look-up
crack_hashes(df)

In [7]:
# open the partial mapping table
partial_mapping = {}
with open('data/partial_mapping.pickle', 'rb') as handle:
    partial_mapping = pk.load(handle)

In [9]:
# visualize the partial mapping table
df_map = pd.DataFrame(partial_mapping.items(),columns=["Pseudonym", "E-mail"])
df_map

,Pseudonym,E-mail
0,6098239d3dff4173859757b08efbcd394444684e87800b...,lucifero.marazzina@studium.unict.it
1,f87649a54babb5ece11636e038b86f91d978bfc13042d5...,adilia.ceola@studium.unict.it
2,ecd1c553df13a400324db32ef72dd652784ee03bbc6cd3...,ricordano.picollo@studium.unict.it
